<a href="https://colab.research.google.com/github/Bigdata92/190902AdvancedVisualPerception/blob/master/190910_Boston_House_price_prediction_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import keras
keras.__version__



'2.2.5'

In [0]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

In [63]:
print(train_data.shape, test_data.shape)
print(train_targets[0])
print(train_targets[0])

(404, 13) (102, 13)
15.2
15.2


여기서 볼 수 있듯이 404개의 훈련 샘플과 102개의 테스트 샘플이 있고 모두 13개의 수치 특성을 가지고 있습니다. 13개의 특성은 다음과 같습니다:

1. Per capita crime rate.
2. Proportion of residential land zoned for lots over 25,000 square feet.
3. Proportion of non-retail business acres per town.
4. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5. Nitric oxides concentration (parts per 10 million).
6. Average number of rooms per dwelling.
7. Proportion of owner-occupied units built prior to 1940.
8. Weighted distances to five Boston employment centres.
9. Index of accessibility to radial highways.
10. Full-value property-tax rate per $10,000.
11. Pupil-teacher ratio by town.
12. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
13. % lower status of the population.

타깃은 주택의 중간 가격으로 천달러 단위입니다:

In [64]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

Data Normalization 

In [0]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std 

test_data -= mean
test_data /= std

Build Model

In [0]:
from keras import models
from keras import layers

def build_model():
  # 동일한 모델을 여러 번 생성할 것이므로 함수를 만들어 사용합니다 
  model = models.Sequential()
  model.add(layers.Dense(64, activation = 'relu', input_shape = (train_data.shape[1],))) # train_data.shape = (404, 13)이므로 train_data.shape[1] = 13, 1d 이므로 ,뒤에 비워놓으면 알아서 1됨
  model.add(layers.Dense(64, activation = 'relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer = 'rmsprop', loss='mse', metrics = ['mae']) 
  
  return model


마지막 레어에는 Activation function이 없습니다. 회귀 문제(어떤 값을 예측하는 문제)를 풀때 이와 같이 구성합니다.
평균 제곱 오차 (mean squared error)
절대 평균 오차 (mean absolute error)

0.5 indicate $500

**K-Fold Validation** 

In [68]:
import numpy as np

k = 4 # k등분 train= {1,2,3,4}, test={5}, train = {1,2,3,5}, test = {4} ~~ 0.83, 0.87, 0.86, 0.84, 0.87 평균내서 평균 모델의 성능 나타냄
num_val_samples = len(train_data) // k # 1덩어리 
num_epochs = 100
all_scores = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size=1, verbose=1)
    # 검증 세트로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 1)
    all_scores.append(val_mae)


처리중인 폴드 # 0
Epoch 1/100
303/303 [==============================] - 1s 4ms/step - loss: 230.8951 - mean_absolute_error: 11.4746
Epoch 2/100
303/303 [==============================] - 1s 4ms/step - loss: 32.6984 - mean_absolute_error: 3.9106
Epoch 3/100
303/303 [==============================] - 1s 4ms/step - loss: 22.2147 - mean_absolute_error: 3.2513
Epoch 4/100
303/303 [==============================] - 1s 3ms/step - loss: 18.8224 - mean_absolute_error: 2.9653
Epoch 5/100
303/303 [==============================] - 1s 3ms/step - loss: 16.4237 - mean_absolute_error: 2.7054
Epoch 6/100
303/303 [==============================] - 1s 3ms/step - loss: 14.7240 - mean_absolute_error: 2.5427
Epoch 7/100
303/303 [==============================] - 1s 3ms/step - loss: 13.9038 - mean_absolute_error: 2.4233
Epoch 8/100
303/303 [==============================] - 1s 3ms/step - loss: 13.3317 - mean_absolute_error: 2.4696
Epoch 9/100
303/303 [==============================] - 1s 3ms/step - loss: 12.9313

In [70]:
all_scores

[2.253315536102446, 2.6723119480775135, 2.5963974801620635, 2.3301143056095235]

In [71]:
np.mean(all_scores)

2.4630348174878867

In [0]:
# 아래 모델은 k fold cross validation에서 마지막에 훈련된 모델입니다.
# 본 실습에서 별도의 test data set을 구성하지 않았기 때문에, vadliation data set을 이용해 house price 를 예측prediction 해보겠습니다.

predictedHousePrice = model.predict(val_data)

In [73]:
# validation set 은 100개의 데이터 셋으로 구성되어 있습니다만 아래와 같이 1대1 비교를 해보겠습니다.
# val_data[0] 에 대한 예측값 predictedHousePrice[0]
# val_data[0] 에 대한 실제값 val_targets[0]
print (predictedHousePrice[0], val_targets[0])

[24.867008] 36.2
